# CloudClient Walkthrough

This notebook is intended to walk a new user through a full workflow using `cfa-cloudops`. It will take you from initializing the client and uploading files to Blob Storage to creating a pool and running a job.

## Import and Initialize

In [ ]:
# import the cloudclient class
from cfa.cloudops import CloudClient

The initialization below is the simplest way to create and instance of the `CloudClient` class. It will use environment variables or values stored in a .env file to authenticate, like the .env file stored [here](../../files/sample.env), and a managed identity credential based on your local working environment. The .env file should be stored at the same level in the directory in which you're working.

In [ ]:
# initialize
cc = CloudClient()

There are other methods of intializing the CloudClient, such as pointing to a different location of the .env file (perhaps it's called my_azure.env), or using a service principal or federated token for the credentials.

In [ ]:
## use a different path to .env file - uncomment if desired to run
# cc = CloudClient(dotenv_path = 'my_azure.env')

## use a service principal instead
# cc = CloudClient(use_sp = True)


## use a federated token
# cc = CloudClient(use_federated = True)

## Upload to Blob Storage

There are plenty of times when local files would need to be uploaded to Blob Storage. Files can be referenced from within a running job via a mount in the pool. Scripts in Blob Storage can also be referenced in the command line for the task execution.

For example, we have the `main.py` file that we want to upload to the Blob container 'input-test' in order to use it for a future task. The following code will upload to the root of the specified container.

In [ ]:
cc.upload_files(
    "main.py",
    container_name = "input-test"
)

## Upload Image to Container Registry

Batch pools can use images from Azure Container Registry, GitHub Container Registry, or Docker Hub. Suppose we want to package the local Dockerfile (python image with a few requirements) and upload to the Azure Container Registry for use by the pool. The following code would do the trick. Make sure to reference the correct registry name.

In [ ]:
container_name = cc.package_and_upload_dockerfile(
    registry_name = "my_azure_registry",
    repo_name = "simple_test",
    tag = "latest"
)

## Create a Pool

Pools are usually created for each team or per project. It spins up nodes when necessary based on the container you specify. The following would create a pool based on the Docker image we just uploaded, autoscaling to 5 nodes, mounting to the 'input-test' container we uploaded to (with relative mount path 'inputs'), an 8 core CPU, and call it 'getting-started-pool'. 

In [ ]:
cc.create_pool(
    "getting-started-pool",
    mounts = [('input-test', 'inputs')],
    container_image_name = container_name,
    vm_size = "standard_d8s_v3",
    max_autoscale_nodes = 5
)

## Create a Job

Now we can create a job to run our set of tasks. Let's call it 'getting-started-job'.

In [ ]:
cc.create_job(
    "getting-started-job",
    pool_name = "getting-started-pool",
    exist_ok = True
)

## Add Tasks to Job

At this point we are ready to add tasks to the job we created. We can run the `main.py` python script that we uploaded to the 'input-test' container. It takes an argument called '--user' and prints a welcome message to the console. We will add two tasks to our job for two different users. In general, any number of tasks can be added to a job.

In [ ]:
cc.add_task(
    job_name = "getting-started-job",
    command_line = "python3 /inputs/main.py --user Ryan"
)

cc.add_task(
    job_name = "getting-started-job",
    command_line = "python3 /inputs/main.py --user Phil"
)

## Monitor the Job

Now that a job is created and tasks are added to it, we can monitor it locally in our terminal. 

In [ ]:
cc.monitor_job(
    "getting-started-job"
)

## Delete Services

Suppose we want to delete the job and the pool now that the job has completed. The following would remove both from Azure.

In [ ]:
# delete the job
cc.delete_job("getting-started-job")

# delete the pool
cc.delete_pool("getting-started-pool")